---
---
# BOSCH AI HACKATHON 2021
---
## Team - KanthabAI

Members:
  - Sanjana  
  - Lakshya  
  - Abeesh  
  - Sachin  
  - Shubham
  
~ Code, data and scripts maintained by Lakshya Dev at https://github.com/lakshyads/bosch-ai-hackathon-2021-kanthabai

  
---
**Content:**  

1. Initial Config   
2. Prep the environment and data   
3. ***Detecting new Images***   
4. Training the Yolov5 model   
5. Running inferences on training data   
6. Test with test data   
7. Retraining from last checkpoint   
---
---

## Initial Configurations

In [ ]:
available_models = {
    1: "yolov5s.yaml",
    2: "yolov5m.yaml",
    3: "yolov5l.yaml",
    4: "yolov5x.yaml",
    5: "yolov5s6.yaml",
    6: "yolov5m6.yaml",
    7: "yolov5l6.yaml",
    8: "yolov5x6.yaml"
}

selected_model = available_models[7] # Edit this to change the selected model variant of Yolov5
weights_path = 'yolov5l6.pt'
epochs = 350
batch_size = 64
image_size = 640
# Github pull repo and branch
git_branch = 'yolo-l6'
git_token = input('Enter github access token to commit/push your training/test/validation runs back to the selected branch. Leave blank to omit : ')

# Detecting on new images (set archive path if archived folder else set images path)
detect_images_archive_path = ''
detect_images_path = ''

# Testing Or Detecting on images - set to 'latest' to use last trained weights or set custom path
weights_path_for_testing_or_detecting = 'latest'

---
# 1. PREP THE ENVIRONMENT AND DATA
---

### Setup Git & LFS for downloading repo & dataset & ready the model/env

In [ ]:
# Setup
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git config --global user.email "lakshyadev@live.com"
!git config --global user.name "Lakshya from google colab"

# Clone repo
!git clone https://github.com/lakshyads/bosch-ai-hackathon-2021-kanthabai.git yolov5
%cd /content/yolov5/
!git pull
!git checkout $git_branch

# Install dependencies and give max permissions
!pip install -r requirements.txt
%cd /content/yolov5/
!chmod -R 755 /content/yolov5
# Download pretrained weights
!/content/yolov5/weights/download_weights.sh

---
# 2. DETECTING ON NEW IMAGES
---

### Extract new images from zipped upload

In [ ]:
%cd ..
!unzip -q $detect_images_archive_path ./detection_images
%cd yolov5

### Runs the detect script.
### Update --weights with required weights path

In [ ]:
use_weights = ''
if (weights_for_testing_path == 'latest'):
  from utils.plots import plot_results
  import os
  # find the latest run
  _, dir_names, _ = next(os.walk('runs/train'))
  dir_names.sort()
  latest = os.path.join(dirPath, dir_names[-1]) # eg: 'runs/train/exp3'
  _, _, latest_run_weights = next(os.walk('latest/weights'))
  if ('best.pt' not in set(latest_run_weights)):
      use_weights = f'{latest}/weights/last.pt'
  else:
        use_weights = f'{latest}/weights/best.pt'
else:
  use_weights = weights_path_for_testing_or_detecting

##########################################################
!python detect.py --source $detect_images_path --weights $use_weights --img $image_size --save-txt --save-conf


---
# 3. TRAINING THE YOLO-V5 MODEL
---

### Ready the data set included in the above repo 
#### (Extract and create train & val sets. Set 'make_test_set' to True below to create test set also.)

In [ ]:

# Set to True to create test set
make_test_set = False

# =================================================
%cd /content/yolov5/
# Unzip dataset
!unzip -q ./data/final-data-full.zip -d /content/yolov5/data/dataset/
# =================================================

if(make_test_set is False):
  #split dataset without test set
  !python /content/yolov5/data-utils/split_train_val_test.py --images /content/yolov5/data/dataset --labels /content/yolov5/data/dataset --out /content/yolov5/data/dataset --move y --test n
else:
  #split dataset with test set
  !python /content/yolov5/data-utils/split_train_val_test.py --images /content/yolov5/data/dataset --labels /content/yolov5/data/dataset --out /content/yolov5/data/dataset --move y --test y

### Run Training

In [ ]:
if (weights_path is not None and len(weights_path) > 0):
    !python train.py --img $image_size --batch $batch_size --epochs $epochs --data final-data.yaml --cfg $selected_model --weights $weights_path --nosave --cache 
else:
    !python train.py --img $image_size --batch $batch_size --epochs $epochs --data final-data.yaml --cfg $selected_model --weights '' --nosave --cache

### Commit runs/train folder updates to git

In [ ]:

if (git_token is not None and len(git_token) > 0):
    import os

    # find the latest run
    dirPath, dir_names, _ = next(os.walk('runs/train'))
    dir_names.sort()
    latest = os.path.join(dirPath, dir_names[-1]) # eg: 'runs/train/exp3'

    !git lfs track runs/
    !git add runs/train
    !git commit -m "Updated $latest data after a new training execution"
    !git push https://$git_token@github.com/lakshyads/bosch-ai-hackathon-2021-kanthabai.git
else:
    print("No valid Github access token found to commit or push changes")

---
# 4. RUNNING INFERENCES ON TRAINING DATA
---

### Initialize Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/train

### Plotting the training results from latest run

In [ ]:
from utils.plots import plot_results
import os

# find the latest run
_, dir_names, _ = next(os.walk('runs/train'))
dir_names.sort()

latest = os.path.join(dirPath, dir_names[-1]) # eg: 'runs/train/exp3'

plot_results(save_dir=f'{latest}')

---
# 5. TESTING
---

### Update --weights with required weights path

In [ ]:
use_weights = ''
if (weights_for_testing_path == 'latest'):
  from utils.plots import plot_results
  import os
  # find the latest run
  _, dir_names, _ = next(os.walk('runs/train'))
  dir_names.sort()
  latest = os.path.join(dirPath, dir_names[-1]) # eg: 'runs/train/exp3'
  _, _, latest_run_weights = next(os.walk('latest/weights'))
  if ('best.pt' not in set(latest_run_weights)):
      use_weights = f'{latest}/weights/last.pt'
  else:
        use_weights = f'{latest}/weights/best.pt'
else:
  use_weights = weights_path_for_testing_or_detecting

###############################################################33
!python test.py --weights $use_weights --data final-data.yaml --img $image_size


### Commit runs/test folder updates to git

In [ ]:
if (git_token is not None and len(git_token) > 0):
    import os

    # find the latest run
    dirPath, dir_names, _ = next(os.walk('runs/train'))
    dir_names.sort()
    latest = os.path.join(dirPath, dir_names[-1]) # eg: 'runs/test/exp3'

    !git lfs track runs/
    !git add runs/train
    !git commit -m "Updated $latest data after a new testing execution"
    !git push https://$git_token@github.com/lakshyads/bosch-ai-hackathon-2021-kanthabai.git
else:
    print("No valid Github access token found to commit or push changes")